In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import requests
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [26]:
def produce_sample(case_df_by_state):
    data_array = None
    for state in case_states:
        if state in vacc_states:
            this_past_df = pd.DataFrame({})

            this_case_df = case_df_by_state[[state]]
            this_case_df.rename(columns={state:'case_ma'},inplace=True)
            this_vacc_df = vacc_df_by_state[[state]]
            this_vacc_df.rename(columns={state:'vacc_rate'},inplace=True)
            this_df = pd.concat([this_case_df,this_vacc_df],axis=1)

            # this_df.to_csv('./data_by_state/'+state+'.csv',index=True)

            this_past_df['case_ma'] = this_df.case_ma
            this_past_df['vacc_rate'] = this_df.vacc_rate

            for i in range(1,21):
                this_past_df['_'.join(['case',str(i),'day_ago'])] = this_df.case_ma.shift(periods=i)
            for i in range(1,21):
                this_past_df['_'.join(['vacc',str(i),'day_ago'])] = this_df.vacc_rate.shift(periods=i)
            this_past_df.dropna(inplace=True)

            if data_array is None:
                data_array = this_past_df.to_numpy()
            else:
                data_array = np.concatenate((data_array,this_past_df.to_numpy()),axis=0)
    
    X = data_array[:,2:]
    y = data_array[:,0]
    y =  y.reshape(-1,1)

    return X,y

In [2]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [3]:
url="https://static.usafacts.org/public/data/covid-19/covid_confirmed_usafacts.csv?_ga=2.135985718.681365941.1640147350-1281724743.1639242218"
s=requests.get(url).content
case_df=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [4]:
url="https://static.usafacts.org/public/data/external/COVID19_CDC_Vaccination_CSV_Download.csv?_ga=2.232519448.681365941.1640147350-1281724743.1639242218"
s=requests.get(url).content
vacc_df=pd.read_csv(io.StringIO(s.decode('utf-8')))

In [5]:
case_states = list(pd.value_counts(case_df.State).keys())
case_df_by_state = pd.DataFrame({})
for state in case_states:
    this_df = case_df[case_df.State == state]
    this_df = this_df.iloc[:,4:]
    this_ts = this_df.sum(axis=0).diff(periods=1).rolling(window=7).mean()
    this_df = this_ts.to_frame(name=abbrev_to_us_state[state])
    case_df_by_state = pd.concat([case_df_by_state,this_df],axis=1)
case_df_by_state

,Texas,Georgia,Virginia,Kentucky,Missouri,Kansas,Illinois,North Carolina,Iowa,Tennessee,...,Maine,Arizona,Vermont,Massachusetts,New Hampshire,Connecticut,Rhode Island,Hawaii,Delaware,District of Columbia
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-16,7433.000000,1323.857143,2673.142857,2152.142857,2829.428571,1608.571429,8009.428571,3201.142857,1496.571429,2063.857143,...,1048.000000,3092.571429,430.285714,4449.571429,1446.000000,2568.571429,1051.285714,230.714286,631.571429,273.571429
2021-12-17,4635.285714,1429.285714,2760.142857,2111.714286,2793.428571,1668.571429,8463.714286,3256.285714,1013.428571,1793.857143,...,905.428571,3058.571429,393.142857,4644.000000,1228.571429,2306.142857,1068.285714,320.285714,679.000000,359.571429
2021-12-18,5107.000000,1429.285714,2760.142857,2111.714286,2793.428571,1668.571429,8739.714286,3256.285714,1013.428571,1793.857143,...,914.714286,3058.571429,418.571429,4644.000000,1228.571429,2306.142857,1068.285714,320.285714,679.000000,359.571429
2021-12-19,5789.142857,1429.285714,2760.142857,2111.714286,2793.428571,1668.571429,9069.571429,3256.285714,1013.428571,1793.857143,...,914.857143,3058.571429,406.571429,4644.000000,1228.571429,2306.142857,1068.285714,320.285714,679.000000,359.571429


In [6]:
vacc_df = vacc_df[vacc_df.GEOGRAPHY_LEVEL == 'State']

In [7]:
pd.value_counts(vacc_df.GEOGRAPHY_LEVEL)

State    18838
Name: GEOGRAPHY_LEVEL, dtype: int64

In [8]:
vacc_df.sort_values(['GEOGRAPHY_NAME','DATE'], ascending=True,inplace=True)

In [9]:
vacc_df.head()

,DATE,GEOGRAPHY_LEVEL,GEOGRAPHY_NAME,DEMOGRAPHIC_GROUP,DEMOGRAPHIC_CATEGORY,PARTIALLY_OR_FULLY_VACCINATED_PERSONS,FULLY_VACCINATED_PERSONS,POPULATION,PARTIALLY_OF_FULLY_VACCINATED_PERCENT,FULLY_VACCINATED_PERCENT,TOTAL_DOSES_ADMINISTERED,TOTAL_DOSES_DISTRIBUTED,SOURCEINFO,USAFACTS_INGESTION_DATE
306,2021-01-19,State,Alabama,Total,Total,114319.0,16346.0,4903185.0,0.023315,0.003334,130795.0,444650.0,CDC State and National Vaccination Data,2021-01-20T10:04:32.264Z
305,2021-01-20,State,Alabama,Total,Total,121113.0,17956.0,4903185.0,0.024701,0.003662,139200.0,483275.0,CDC State and National Vaccination Data,2021-01-20T14:00:25.221Z
304,2021-01-21,State,Alabama,Total,Total,144429.0,21345.0,4903185.0,0.029456,0.004353,165919.0,493125.0,CDC State and National Vaccination Data,2021-02-01T12:21:47.453Z
303,2021-01-22,State,Alabama,Total,Total,174317.0,24840.0,4903185.0,0.035552,0.005066,199314.0,519525.0,CDC State and National Vaccination Data,2021-02-01T12:22:55.206Z
302,2021-01-23,State,Alabama,Total,Total,193955.0,27422.0,4903185.0,0.039557,0.005593,221540.0,521225.0,CDC State and National Vaccination Data,2021-02-01T12:24:00.052Z


In [10]:
vacc_df_by_state = pd.DataFrame({})
vacc_states = list(pd.value_counts(vacc_df.GEOGRAPHY_NAME).keys())
for state in vacc_states:
    this_df = vacc_df[vacc_df.GEOGRAPHY_NAME == state]
    this_df = this_df[['DATE','FULLY_VACCINATED_PERCENT']]
    this_df.set_index(keys=['DATE'],inplace=True)
    this_df.rename(columns={'FULLY_VACCINATED_PERCENT':state},inplace=True)
    vacc_df_by_state = pd.concat([vacc_df_by_state,this_df],axis=1)
vacc_df_by_state.sort_index(axis=0,inplace=True)
vacc_df_by_state

,Alaska,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York State,North Carolina,...,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Marshall Islands,Maryland,Massachusetts,Wyoming
DATE,,,,,,,,,,,,,,,,,,,,,
2021-01-19,0.014766,0.007392,0.008113,0.007165,0.005410,0.005236,0.004940,0.011081,0.005253,0.005150,...,0.007555,0.004003,0.005355,0.002890,0.007307,0.007453,0.000034,0.003605,0.005956,0.005602
2021-01-20,0.015424,0.008038,0.008534,0.007477,0.005587,0.005893,0.005564,0.011783,0.005491,0.005569,...,0.008707,0.004436,0.005496,0.003095,0.007539,0.007931,0.000034,0.004028,0.006103,0.005945
2021-01-21,0.016885,0.009578,0.009321,0.008378,0.006265,0.007589,0.005564,0.012883,0.006210,0.006363,...,0.009923,0.005344,0.005983,0.003514,0.008195,0.008980,NaN,0.004649,0.006710,0.005945
2021-01-22,0.018752,0.010868,0.011114,0.009350,0.006605,0.009248,0.006329,0.013963,0.007300,0.007096,...,0.010990,0.007109,0.006633,0.005051,0.008687,0.011079,NaN,0.005107,0.007495,0.007801
2021-01-23,0.020786,0.011155,0.012383,0.010156,0.006809,0.009683,0.006925,0.015801,0.008363,0.007551,...,0.011584,0.008376,0.007168,0.005722,0.008989,0.013065,NaN,0.005639,0.008396,0.008247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-24,0.540177,0.507907,0.517209,0.571975,0.542390,0.646320,0.675192,0.633339,0.682040,0.539691,...,0.505256,0.565048,0.544025,0.519028,0.486611,0.720536,0.280161,0.671794,0.708492,0.453413
2021-11-29,0.540747,0.508426,0.518238,0.572025,0.544343,0.647458,0.677514,0.634756,0.684396,0.540689,...,0.505657,0.565106,0.544704,0.520928,0.488744,0.721358,0.283325,0.674703,0.711068,0.455205
2021-11-30,0.542261,0.509334,0.518636,0.574070,0.545035,0.647830,0.678163,0.635171,0.684952,0.542146,...,0.505789,0.567341,0.546495,0.521277,0.488873,0.722957,0.283325,0.675594,0.711846,0.455739


In [16]:
vacc_i, vacc_f = tuple(np.array(list(vacc_df_by_state.index))[[1,-1]])
case_i, case_f = tuple(np.array(list(case_df_by_state.index))[[1,-1]])

In [19]:
case_states = case_df_by_state.columns.tolist()

In [21]:
case_df_by_state_minmax = pd.DataFrame(MinMaxScaler().fit_transform(case_df_by_state), columns=case_df_by_state.columns,index=case_df_by_state.index)
case_df_by_state_standa = pd.DataFrame(StandardScaler().fit_transform(case_df_by_state), columns=case_df_by_state.columns,index=case_df_by_state.index)

In [22]:
case_df_by_state_minmax

,Texas,Georgia,Virginia,Kentucky,Missouri,Kansas,Illinois,North Carolina,Iowa,Tennessee,...,Maine,Arizona,Vermont,Massachusetts,New Hampshire,Connecticut,Rhode Island,Hawaii,Delaware,District of Columbia
2020-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-16,0.322563,0.178924,0.433550,0.398440,0.541939,0.421076,0.680648,0.299208,0.288651,0.172314,...,0.976701,0.210610,0.631579,0.541617,0.808144,0.851245,0.697402,0.249923,0.763690,0.760826
2021-12-17,0.201153,0.193173,0.447660,0.390955,0.535524,0.436782,0.713812,0.304362,0.195465,0.149772,...,0.843829,0.208295,0.577060,0.563117,0.686627,0.764274,0.707812,0.346951,0.821040,1.000000
2021-12-18,0.221624,0.193173,0.447660,0.390955,0.535524,0.436782,0.733960,0.304362,0.195465,0.149772,...,0.852483,0.208295,0.614385,0.563117,0.686627,0.764274,0.707812,0.346951,0.821040,1.000000
2021-12-19,0.251226,0.193173,0.447660,0.390955,0.535524,0.436782,0.758041,0.304362,0.195465,0.149772,...,0.852616,0.208295,0.596771,0.563117,0.686627,0.764274,0.707812,0.346951,0.821040,1.000000


In [24]:
X, y = produce_sample(case_df_by_state_minmax)

C:\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
np.save('./feature_minmax.npy',X)
np.save('./target_minmax.npy', y)

In [61]:
X, y = produce_sample(case_df_by_state_standa)
np.save('./feature_standa.npy',X)
np.save('./target_standa.npy', y)

C:\Python39\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [62]:
X.shape

(7101, 40)

In [20]:
X = data_array[:,2:]
X.shape

(7101, 40)

In [21]:
X[0]

array([1.75871429e+04, 1.82344286e+04, 1.89797143e+04, 1.96344286e+04,
       2.00540000e+04, 2.03387143e+04, 2.07997143e+04, 1.71580000e+04,
       1.71687143e+04, 1.69627143e+04, 1.74727143e+04, 1.80641429e+04,
       1.97272857e+04, 1.73275714e+04, 1.80805714e+04, 1.87715714e+04,
       1.97694286e+04, 2.04205714e+04, 2.07012857e+04, 1.99622857e+04,
       2.68954408e-02, 2.53272525e-02, 2.31336996e-02, 2.14510123e-02,
       1.94659717e-02, 1.73401870e-02, 1.73401870e-02, 1.65131041e-02,
       1.55898005e-02, 1.42725099e-02, 1.28854164e-02, 1.11813123e-02,
       1.04043743e-02, 9.31573695e-03, 8.93423449e-03, 8.68450936e-03,
       7.95271577e-03, 7.08852406e-03, 6.46653916e-03, 6.20091523e-03])

In [19]:
y = data_array[:,0]
y =  y.reshape(-1,1)
y.shape

(10751, 1)

In [20]:
y

array([[16962.71428571],
       [17168.71428571],
       [17158.        ],
       ...,
       [   57.85714286],
       [  103.71428571],
       [  102.42857143]])

In [21]:
np.save('./feature.npy',X)
np.save('./target.npy', y)

In [22]:
X = np.load('./feature.npy')
y = np.load('./target.npy')

In [23]:
X

array([[1.74727143e+04, 1.80641429e+04, 1.97272857e+04, ...,
        7.08852406e-03, 6.46653916e-03, 6.20091523e-03],
       [1.69627143e+04, 1.74727143e+04, 1.80641429e+04, ...,
        7.95271577e-03, 7.08852406e-03, 6.46653916e-03],
       [1.71687143e+04, 1.69627143e+04, 1.74727143e+04, ...,
        8.68450936e-03, 7.95271577e-03, 7.08852406e-03],
       ...,
       [9.38571429e+01, 9.38571429e+01, 9.38571429e+01, ...,
        6.34734162e-01, 6.34250987e-01, 6.33621868e-01],
       [5.78571429e+01, 9.38571429e+01, 9.38571429e+01, ...,
        6.35305186e-01, 6.34734162e-01, 6.34250987e-01],
       [1.03714286e+02, 5.78571429e+01, 9.38571429e+01, ...,
        6.35710430e-01, 6.35305186e-01, 6.34734162e-01]])

In [24]:
y

array([[16962.71428571],
       [17168.71428571],
       [17158.        ],
       ...,
       [   57.85714286],
       [  103.71428571],
       [  102.42857143]])